In [ ]:
from collections import defaultdict
import random
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import torch
from research.utils.data_access_utils import S3AccessUtils, RDSAccessUtils
from research.weight_estimation.weight_estimator import WeightEstimator
from research.weight_estimation.biomass_estimator import NormalizeCentered2D, NormalizedStabilityTransform, ToTensor, Network
from research.gtsf_data.gtsf_dataset import GTSFDataset
from research.gtsf_data.body_parts import BodyParts
from research.utils.keypoint_transformations import get_keypoint_arr
from research.weight_estimation.optics import pixel2world
from scipy.spatial.distance import pdist

pd.set_option('display.max_rows', 500)

In [ ]:
akpd_scorer_url = 'https://aquabyte-models.s3-us-west-1.amazonaws.com/keypoint-detection-scorer/akpd_scorer_model_TF.h5'
gtsf_dataset = GTSFDataset('2019-02-01', '2020-03-30', akpd_scorer_url)
df = gtsf_dataset.get_prepared_dataset()

In [ ]:
BODY_PARTS = BodyParts().get_core_body_parts()

In [ ]:
tdf = df[(df.median_depth < 0.75) & (df.akpd_score > 0.5) & (df.captured_at < '2019-09-27')].copy(deep=True)

In [ ]:
analysis_data = defaultdict(list)

# instantiate weight estimator class
model_f = '/root/data/alok/biomass_estimation/playground/nn_epoch_253.pb'
weight_estimator = WeightEstimator(model_f)


count = 0
for idx, row in tdf.iterrows():
    if count % 1000 == 0:
        print(count)
    count += 1
    world_keypoints = row.world_keypoints
    cm = row.camera_metadata
    weight = row.weight
    
    for t in range(10):
        random_x_addition = np.random.uniform(-0.3, 0.3)
        random_y_addition = np.random.uniform(-0.3, 1.5)
        random_z_addition = np.random.uniform(-0.3, 0.3)
        
        new_world_keypoints = {
            body_part: np.array([item[0] + random_x_addition, 
                                 item[1] + random_y_addition, 
                                 item[2] + random_z_addition])
            for body_part, item in world_keypoints.items()}

        new_ann_left, new_ann_right = [], []
        for body_part in BODY_PARTS:
            # add left item
            x, y, z = new_world_keypoints[body_part]
            x_frame_l = x * cm['focalLengthPixel'] / y + cm['pixelCountWidth'] // 2
            y_frame_l = -z * cm['focalLengthPixel'] / y + cm['pixelCountHeight'] // 2
            item_l = {
                'keypointType': body_part,
                'xFrame': x_frame_l,
                'yFrame': y_frame_l
            }
            new_ann_left.append(item_l)

            # add right item
            disparity = cm['focalLengthPixel'] * cm['baseline'] / y
            x_frame_r = x_frame_l - disparity
            y_frame_r = y_frame_l
            item_r = {
                'keypointType': body_part,
                'xFrame': x_frame_r,
                'yFrame': y_frame_r
            }
            new_ann_right.append(item_r)

        new_ann = {
            'leftCrop': new_ann_left,
            'rightCrop': new_ann_right
        }
        new_world_keypoints = pixel2world(new_ann['leftCrop'], new_ann['rightCrop'], cm)
        new_median_x = np.median([wkp[0] for wkp in new_world_keypoints.values()])
        new_median_y = np.median([wkp[1] for wkp in new_world_keypoints.values()])
        new_median_z = np.median([wkp[2] for wkp in new_world_keypoints.values()])

        weight_prediction = weight_estimator.predict(new_ann, camera_metadata)
        input_sample = {
            'keypoints': new_ann,
            'cm': cm,
            'stereo_pair_id': row.id,
            'single_point_inference': True
        }
        nomralized_centered_2D_kps = \
        normalize_centered_2D_transform.__call__(input_sample)

        normalized_stability_kps = normalized_stability_transform.__call__(nomralized_centered_2D_kps)
        tensorized_kps = to_tensor_transform.__call__(normalized_stability_kps)
        original_weight_prediction = network(tensorized_kps['kp_input']).item() * 1e4
        

        analysis_data['new_ann_list'].append(new_world_keypoints)
        analysis_data['new_median_x'].append(new_median_x)
        analysis_data['new_median_y'].append(new_median_y)
        analysis_data['new_median_z'].append(new_median_z)
        analysis_data['weight'].append(weight)
        analysis_data['weight_prediction'].append(weight_prediction)
        analysis_data['original_weight_prediction'].append(original_weight_prediction)
        analysis_data['error_pct'].append((weight_prediction - weight) / weight)
        analysis_data['original_error_pct'].append((original_weight_prediction - weight) / weight)





In [ ]:
analysis_df = pd.DataFrame(analysis_data)

In [ ]:
x_values = np.arange(-0.4, 0.4, 0.1)
for idx in range(len(x_values) - 1):
    x_low, x_high = x_values[idx], x_values[idx + 1]
    x_mask = (analysis_df.new_median_x > x_low) & (analysis_df.new_median_x < x_high)
    y_mask = (analysis_df.new_median_y > 1.8)# & (analysis_df.new_median_y < 1.3)
    mask = x_mask * y_mask
    error_pct = 100 * (analysis_df[mask].weight_prediction.mean() - analysis_df[mask].weight.mean()) / \
                analysis_df[mask].weight.mean()
    print('X Range: {} <-> {}, Deviation: {}%'.format(round(x_low, 2), 
                                                      round(x_high, 2),
                                                      round(error_pct, 2)))

In [ ]:
z_values = np.arange(-0.4, 0.4, 0.1)
for idx in range(len(z_values) - 1):
    z_low, z_high = z_values[idx], z_values[idx + 1]
    z_mask = (analysis_df.new_median_z > z_low) & (analysis_df.new_median_z < z_high)
    y_mask = (analysis_df.new_median_y > 1.8)# & (analysis_df.new_median_y < 1.3)
    mask = z_mask * y_mask
    error_pct = 100 * (analysis_df[mask].weight_prediction.mean() - analysis_df[mask].weight.mean()) / \
                analysis_df[mask].weight.mean()
    print('X Range: {} <-> {}, Deviation: {}%'.format(round(x_low, 2), 
                                                      round(x_high, 2),
                                                      round(error_pct, 2)))

In [ ]:
x_values = np.arange(-0.4, 0.4, 0.1)
for idx in range(len(x_values) - 1):
    x_low, x_high = x_values[idx], x_values[idx + 1]
    x_mask = (analysis_df.new_median_x > x_low) & (analysis_df.new_median_x < x_high)
    y_mask = (analysis_df.new_median_y > 1.8)# & (analysis_df.new_median_y < 1.3)
    mask = x_mask * y_mask
    error_pct = 100 * (analysis_df[mask].original_weight_prediction.mean() - analysis_df[mask].weight.mean()) / \
                analysis_df[mask].weight.mean()
    print('X Range: {} <-> {}, Deviation: {}%'.format(round(x_low, 2), 
                                                      round(x_high, 2),
                                                      round(error_pct, 2)))

In [ ]:
z_values = np.arange(-0.4, 0.5, 0.1)
for idx in range(len(z_values) - 1):
    z_low, z_high = z_values[idx], z_values[idx + 1]
    z_mask = (analysis_df.new_median_z > z_low) & (analysis_df.new_median_z < z_high)
    y_mask = (analysis_df.new_median_y < 1.0)# & (analysis_df.new_median_y < 1.5)
    mask = z_mask * y_mask
    error_pct = 100 * (analysis_df[mask].original_weight_prediction.mean() - analysis_df[mask].weight.mean()) / \
                analysis_df[mask].weight.mean()
    print('Y Range: {} <-> {}, Deviation: {}%'.format(round(z_low, 2), 
                                                      round(z_high, 2),
                                                      round(error_pct, 2)))

In [ ]:
def get_world_keypoints(row):
        return pixel2world(row.new_ann['leftCrop'], row.new_ann['rightCrop'], row.camera_metadata)

df['new_ann'] = new_ann_list
df['new_world_keypoints'] = df.apply(lambda x: get_world_keypoints(x), axis=1)
df['new_median_depth'] = df.new_world_keypoints.apply(lambda x: np.median([wkp[1] for wkp in x.values()]))

In [ ]:
tdf = df[(df.median_depth < 0.75) & (df.akpd_score > 0.5) & (df.captured_at < '2019-09-27')].copy(deep=True)

In [ ]:
# initialize data transforms so that we can run inference with neural network
normalize_centered_2D_transform = NormalizeCentered2D()
normalized_stability_transform = NormalizedStabilityTransform()
to_tensor_transform = ToTensor()

# Get neural network weights from sample training
s3_access_utils = S3AccessUtils('/root/data')
model_url = 'https://aquabyte-models.s3-us-west-1.amazonaws.com/biomass/trained_models/2019-11-08T00-13-09/nn_epoch_798.pb'
model_f, _, _ = s3_access_utils.download_from_url(model_url)
network = torch.load(model_f)

weight_predictions, depths = [], []
for idx, row in tdf.iterrows():
    input_sample = {
        'keypoints': row.new_ann,
        'cm': row.camera_metadata,
        'stereo_pair_id': row.id,
        'single_point_inference': True
    }
    nomralized_centered_2D_kps = \
        normalize_centered_2D_transform.__call__(input_sample)

    normalized_stability_kps = normalized_stability_transform.__call__(nomralized_centered_2D_kps)
    tensorized_kps = to_tensor_transform.__call__(normalized_stability_kps)
    weight_prediction = network(tensorized_kps['kp_input']).item() * 1e4
    weight_predictions.append(weight_prediction)


In [ ]:
tdf['pred_weight'] = weight_predictions

In [ ]:
plt.figure(figsize=(20, 10))
plt.scatter(tdf.weight.values, tdf.pred_weight.values)
plt.plot([0, 10000], [0, 10000], color='red')
plt.xlim([0, 10000])
plt.ylim([0, 10000])
plt.grid()
plt.show()

In [ ]:
(tdf.pred_weight.mean() - tdf.weight.mean()) / tdf.weight.mean()

In [ ]:
depth_values = np.arange(0.5, 2.4, 0.1)
for idx in range(len(depth_values) - 1):
    low_depth, high_depth = depth_values[idx], depth_values[idx + 1]
    mask = (tdf.new_median_depth > low_depth) & (tdf.new_median_depth < high_depth)
    error_pct = 100 * (tdf[mask].pred_weight.mean() - tdf[mask].weight.mean()) / tdf[mask].weight.mean()
    print('Depth Range: {}-{}, Deviation: {}%'.format(round(low_depth, 2), 
                                                     round(high_depth, 2),
                                                     round(error_pct, 2)))

In [ ]:
depth_values = np.arange(0.5, 2.4, 0.1)
for idx in range(len(depth_values) - 1):
    low_depth, high_depth = depth_values[idx], depth_values[idx + 1]
    mask = (tdf.new_median_depth > low_depth) & (tdf.new_median_depth < high_depth)
    error_pct = 100 * (tdf[mask].pred_weight_new.mean() - tdf[mask].weight.mean()) / tdf[mask].weight.mean()
    print('Depth Range: {}-{}, Deviation: {}%'.format(round(low_depth, 2), 
                                                     round(high_depth, 2),
                                                     round(error_pct, 2)))

In [ ]:
# instantiate weight estimator class
model_f = '/root/data/alok/biomass_estimation/playground/nn_epoch_253.pb'
weight_estimator = WeightEstimator(model_f)

# generate sample predictions
weights = []
for idx, row in tdf.iterrows():
    keypoints, camera_metadata = row.new_ann, row.camera_metadata
    weight_prediction = weight_estimator.predict(keypoints, camera_metadata)
    weights.append(weight_prediction)
    if len(weights) % 1000 == 0:
        print(len(weights))

tdf['pred_weight_new'] = weights

In [ ]:
plt.figure(figsize=(20, 10))
plt.scatter(tdf.weight.values, tdf.pred_weight_new.values)
plt.plot([0, 10000], [0, 10000], color='red')
plt.grid()
plt.show()

In [ ]:
(tdf.pred_weight_new.mean() - tdf.weight.mean()) / tdf.weight.mean()

In [ ]:
for idx, row in tdf.head(11).iterrows():
    input_sample = {
        'keypoints': row.keypoints,
        'cm': row.camera_metadata,
        'stereo_pair_id': row.id,
        'single_point_inference': True
    }
    nomralized_centered_2D_kps = \
        normalize_centered_2D_transform.__call__(input_sample)
    normalized_stability_kps = normalized_stability_transform.__call__(nomralized_centered_2D_kps)
    tensorized_kps = to_tensor_transform.__call__(normalized_stability_kps)
    
    keypoint_arr = get_keypoint_arr(row.keypoints, row.camera_metadata, recover_original_depth=True)
    
wkps = tensorized_kps['kp_input'].numpy()[0]
wkp_1 = (0.5 * 0.1) / wkps[:, 2]
x = wkps[:, 0] * wkp_1 / 0.5
y = wkps[:, 1] * wkp_1 / 0.5
plt.figure(figsize=(20, 10))
plt.scatter(x, y, color='blue')

x, y = keypoint_arr[:, 0], keypoint_arr[:, 2]
plt.scatter(x, y, color='red')

plt.grid()
plt.show()

In [ ]:
arr = df.keypoint_arr.iloc[0]
arr -= arr.mean(axis=0)
eigen_values, eigen_vectors = np.linalg.eig(np.dot(arr.T, arr))


In [ ]:
wkps = np.dot(eigen_vectors.T, arr.T).T

In [ ]:
def normalize_3D_coordinates(wkps):

    # translate fish to origin
    v = np.mean(wkps[:8], axis=0)
    wkps -= v

    # perform PCA decomposition and rotate with respect to new axes
    eigen_values, eigen_vectors = np.linalg.eig(np.dot(wkps.T, wkps))
    wkps = np.dot(eigen_vectors.T, wkps.T).T

    return wkps

In [ ]:
normalize_3D_coordinates(df.keypoint_arr.iloc[0])

In [ ]:
def randomly_rotate_and_translate(wkps, random_x_addition, random_y_addition,
                                  random_z_addition, yaw, pitch, roll):

    # convert to radians
    yaw, pitch, roll = [theta * np.pi / 180.0 for theta in [yaw, pitch, roll]]
    
    R_yaw = np.array([
        [np.cos(yaw), -np.sin(yaw), 0],
        [np.sin(yaw), np.cos(yaw), 0],
        [0, 0, 1]
    ])

    R_pitch = np.array([
        [np.cos(pitch), 0, np.sin(pitch)],
        [0, 1, 0],
        [-np.sin(pitch), 0, np.cos(pitch)]
    ])

    R_roll = np.array([
        [1, 0, 0],
        [0, np.cos(roll), -np.sin(roll)],
        [0, np.sin(roll), np.cos(roll)]
    ])

    R = np.dot(R_yaw, (np.dot(R_pitch, R_roll)))
    wkps = np.dot(R, wkps.T).T

    # perform translation
    wkps[:, 0] += random_x_addition
    wkps[:, 1] += random_y_addition
    wkps[:, 2] += random_z_addition
    return wkps

In [ ]:
new_wkps = randomly_rotate_and_translate(wkps, 0.2, 1.0, 0.3, -30, 0, 0)

In [ ]:
new_wkps -= new_wkps.mean(axis=0)
_, eigen_vectors = np.linalg.eig(np.dot(new_wkps.T, new_wkps))


In [ ]:
plt.figure(figsize=(10, 5))
x, y = wkps[:, 0], wkps[:, 2]
plt.scatter(x, y, color='blue')
x, y = new_wkps[:, 0], new_wkps[:, 2]
plt.scatter(x, y, color='red')
plt.grid()
plt.show()

In [ ]:
wkps

In [ ]:
new_wkps

In [ ]:
new_wkps